In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from statistics import mean
from nltk.corpus import wordnet 
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split,cross_val_score
import math
from bs4 import BeautifulSoup
from itertools import combinations
from time import time
from collections import Counter
import operator
import warnings
import operator
import pickle
import numpy as np
import requests
import re
from googlesearch import search

from synonyms import syno
from tf_idf_result import tf_idf_score
from tf_idf_result import gen_vector
from tokenizer import convert_tolowercase
from tokenizer import regextokenizer_func
from cosine_results import cosine_dot
from cosine_results import cosine_similarity
from synonyms import generatesymptoms2


from tf_idf_result import tf_idf_results
from tf_idf_result import diseaseDetail
from cosine_results import cosine_sim_results

warnings.simplefilter("ignore")

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:

df=pd.read_csv("/content/normal (3).csv")
documentname_list=list(df['label_dis'])
df=df.iloc[:,1:]
columns_name=list(df.columns)
documentname_list=list(documentname_list)

N=len(df)
M=len(columns_name)

idf={}
for col in columns_name:
    temp=np.count_nonzero(df[col])
    idf[col]=np.log(N/temp)


tf={}
for i in range(N):
    for col in columns_name:
        key=(documentname_list[i],col)
        tf[key]=df.loc[i,col]

tf_idf={}
for i in range(N):
    for col in columns_name:
        key=(documentname_list[i],col)
        tf_idf[key]=float(idf[col])*float(tf[key])


D = np.zeros((N, M),dtype='float32')
for i in tf_idf:
    sym = columns_name.index(i[1])
    dis=documentname_list.index(i[0])
    D[dis][sym] = tf_idf[i]


In [ ]:
df_comb = pd.read_csv("/content/combinational (3).csv") # Disease combination
df_norm = pd.read_csv("/content/normal (3).csv") # Individual Disease
Y = df_norm.iloc[:, 0:1]
X = df_norm.iloc[:, 1:]

dataset_symptoms = list(X.columns)
diseases = list(set(Y['label_dis']))
diseases.sort()

In [ ]:
splitter = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
user_symptoms = str(input("\nPlease enter symptoms separated by comma(,):\n")).lower().split(',')
processed_user_symptoms=[]
for sym in user_symptoms:
    sym=sym.strip()
    sym=sym.replace('-',' ')
    sym=sym.replace("'",'')
    sym = ' '.join([lemmatizer.lemmatize(word) for word in splitter.tokenize(sym)])
    processed_user_symptoms.append(sym)  


Please enter symptoms separated by comma(,):
cough, bleeding


In [ ]:
user_symptoms = generatesymptoms2(processed_user_symptoms)


After query expansion done by using the symptoms entered
['cough coughing', 'leech hemorrhage run bleeding shed blood haemorrhage bleed phlebotomise phlebotomize']


In [ ]:
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [ ]:

print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)


select_list = input("\nPlease select the relevant symptoms. Enter indices (separated-space):\n").split()


dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))

    
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])
             
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   

Top matching symptoms from your search!
0 : cough
1 : bleeding
2 : coughing

Please select the relevant symptoms. Enter indices (separated-space):
0 1 2


In [ ]:

found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = []    


Common co-occuring symptoms:
0 : fever
1 : shortness breath
2 : headache
3 : runny nose
4 : sore throat
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
no


In [ ]:

k = 10

print("Final list of Symptoms used for prediction are : ")
for val in final_symp:
    print(val)

Final list of Symptoms used for prediction are : 
cough
bleeding
coughing


In [ ]:
# Show top 10 highly probable disease to the user using TF-IDF
tf_idf_results(k,final_symp,tf_idf)




Top 10 diseases predicted based on TF_IDF Matching :

0. Disease : Asthma 	 Score : 4.51
1. Disease : Bang's disease 	 Score : 4.51
2. Disease : Flu 	 Score : 4.51
3. Disease : Disseminated Intravascular Coagulation 	 Score : 4.22
4. Disease : Ebola Virus Disease (EVD) 	 Score : 4.22
5. Disease : Lassa hemorrhagic fever 	 Score : 4.22
6. Disease : Leukemia 	 Score : 4.22
7. Disease : Asbestos-related diseases 	 Score : 3.12
8. Disease : Bronchiolitis 	 Score : 3.12
9. Disease : Common Cold 	 Score : 3.12

More details about the disease? Enter index of disease or '-1' to discontinue:
-1


In [ ]:
# display top k diseases predicted with cosine probablity
cosine_sim_results(k,final_symp,M,columns_name,D,diseases,tf,idf)

Top 10 disease based on Cosine Similarity Matching :
 
0. Disease : Bang's disease 	 Score : 0.65
1. Disease : Lassa hemorrhagic fever 	 Score : 0.54
2. Disease : Flu 	 Score : 0.31
3. Disease : Middle East respiratory syndrome coronavirus (MERS‐CoV) 	 Score : 0.31
4. Disease : Asthma 	 Score : 0.31
5. Disease : Cough 	 Score : 0.28
6. Disease : Nipah virus infection 	 Score : 0.27
7. Disease : Ebola Virus Disease (EVD) 	 Score : 0.26
8. Disease : Leukemia 	 Score : 0.26
9. Disease : Legionnaire's pneumonia 	 Score : 0.24

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1
